## Understanding a flooding event using OPERA DSWx-HLS data


Heavy rains severely impacted Southeast Texas in May 2024 [[1]](https://www.texastribune.org/2024/05/03/texas-floods-weather-harris-county/), resulting in flooding and causing significant damage to property and human life [[2]](https://www.texastribune.org/series/east-texas-floods-2024/). In this notebook, we will retrieve [OPERA DSWx-HLS](https://d2pn8kiwq2w21t.cloudfront.net/documents/ProductSpec_DSWX_URS309746.pdf) data associated to understand the extent of flooding and damage, and visualize data from before, during, and after the event.

In [1]:
import rasterio
import rioxarray
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
import geoviews as gv
from geoviews import opts

import hvplot.xarray  # noqa
import xarray as xr
import xyzservices.providers as xyz

from shapely.geometry import Point
from osgeo import gdal

import pandas as pd

# STAC imports to retrieve cloud data
from pystac_client import Client

from datetime import datetime
import numpy as np

# GDAL setup for accessing cloud data
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF, TIFF')

/home/karthik/mambaforge/envs/climaterisk/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
# Study location
livingston_tx_lonlat = (-95.09, 30.69) # (lon, lat) form

ref_crs = CRS.from_epsg(4326)
dst_crs = CRS.from_epsg(3857)
map_bounds = transform_bounds(ref_crs, dst_crs, *Point(*livingston_tx_lonlat).buffer(10).bounds)

In [3]:
# Visualize location of area of study
livingston_tx = gv.Points([livingston_tx_lonlat])

basemap = gv.tile_sources.OSM
plot = (livingston_tx*basemap).opts(
    opts.Points(
        color='red',
        alpha=0.75,
        size=25,
        width=800,
        height=800,
        xlim=(map_bounds[0], map_bounds[2]),
        ylim=(map_bounds[1], map_bounds[3]))
)
plot

:Overlay
   .Points.I :Points   [Longitude,Latitude]
   .WMTS.I   :WMTS   [Longitude,Latitude]

In [4]:
# The flooding event primarily happened during 04/30 - 05/02
# We will search for data before and after the event
start_date = datetime(year=2024, month=4, day=1)
stop_date = datetime(year=2024, month=5, day=31)
date_range = f'{start_date.strftime("%Y-%m-%d")}/{stop_date.strftime("%Y-%m-%d")}'

# We open a client instance to search for data, and retrieve relevant data records
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'

# Setup PySTAC client
# POCLOUD refers to the PO DAAC cloud environment that hosts earth observation data
catalog = Client.open(f'{STAC_URL}/POCLOUD/') 

# Setup PySTAC client
provider_cat = Client.open(STAC_URL)
catalog = Client.open(f'{STAC_URL}/POCLOUD/')
collections = ["OPERA_L3_DSWX-HLS_V1"]

search_opts = {
    'bbox' : Point(*livingston_tx_lonlat).buffer(0.01).bounds, 
    'collections': collections,
    'datetime' : date_range,
}

search = catalog.search(**search_opts)
results = list(search.items_as_dicts())
print(f"Number of tiles found intersecting given AOI: {len(results)}")

/home/karthik/mambaforge/envs/climaterisk/lib/python3.12/site-packages/pystac_client/client.py:190: NoConformsTo: Server does not advertise any conformance classes.
  warnings.warn(NoConformsTo())


Number of tiles found intersecting given AOI: 95


In [5]:
def search_to_df(results, layer_name):
        '''
        method that takes in search results from a PySTAC Earthdata query and loads the data into a pandas dataframe
        the data frame has columns of date of acquisition, data URLs, and tile ID
        '''

        times = pd.DatetimeIndex([result['properties']['datetime'] for result in results]) # parse of timestamp for each result
        data = {'hrefs': [value['href'] for result in results for key, value in result['assets'].items() if layer_name in key],  # parse out links only to DIST-STATUS data layer
                'tile_id': [value['href'].split('/')[-1].split('_')[3] for result in results for key, value in result['assets'].items() if layer_name in key]}

        # Construct pandas dataframe to summarize granules from search results
        granules = pd.DataFrame(index=times, data=data)
        granules.index.name = 'times'

        return granules

In [6]:
granules = search_to_df(results=results, layer_name='0_B01_WTR')

In [7]:
# We now filter the dataframe to restrict our results to a single tile_id
granules = granules[granules.tile_id == 'T15RTQ']
granules.sort_index(inplace=True)
len(granules)

21

In [8]:
def urls_to_dataset(granule_dataframe):
    '''method that takes in a list of OPERA tile URLs and returns an xarray dataset with dimensions
    latitude, longitude and time'''

    dataset_list = []
    
    for i, row in granule_dataframe.iterrows():
        with rasterio.open(row.hrefs) as ds:
            # extract CRS string
            crs = str(ds.crs).split(':')[-1]

            # extract the image spatial extent (xmin, ymin, xmax, ymax)
            xmin, ymin, xmax, ymax = ds.bounds

            # the x and y resolution of the image is available in image metadata
            x_res = np.abs(ds.transform[0])
            y_res = np.abs(ds.transform[4])

            # read the data 
            img = ds.read()

            # Ensure img has three dimensions (bands, y, x)
            if img.ndim == 2:
                img = np.expand_dims(img, axis=0) 

            

        lon = np.arange(xmin, xmax, x_res)
        lat = np.arange(ymax, ymin, -y_res)

        lon_grid, lat_grid = np.meshgrid(lon, lat)

        da = xr.DataArray(
            data=img,
            dims=["band", "y", "x"],
            coords=dict(
                lon=(["y", "x"], lon_grid),
                lat=(["y", "x"], lat_grid),
                time=i,
                band=np.arange(img.shape[0])
            ),
            attrs=dict(
                description="OPERA DSWx B01",
                units=None,
            ),
        )
        da.rio.write_crs(crs, inplace=True)

        dataset_list.append(da)
    return xr.concat(dataset_list, dim='time').squeeze()

In [9]:
dataset= urls_to_dataset(granules)

In [10]:
COLORS = [(255, 255, 255, 0.1)]*256 # setting all colors to transparent initially
COLORS[0] = (0, 255, 0, 0.1) # Setting not water class to green
COLORS[1] = (0, 0, 255, 1) # Open surface water
COLORS[2] = (0, 0, 255, 1) # Partial surface water

In [11]:
img = dataset.hvplot.quadmesh(title = 'DSWx data for May 2024 Texas floods',
                            x='lon', y='lat', 
                            project=True, rasterize=True,
                            cmap=COLORS, 
                            colorbar=False,
                            widget_location='bottom',
                            tiles = xyz.OpenStreetMap.Mapnik,
                            xlabel='Longitude (degrees)',ylabel='Latitude (degrees)',
                            fontscale=1.25, frame_width=1000, frame_height=1000)

img

BokehModel(combine_events=True, render_bundle={'docs_json': {'6e40716d-3752-46a5-ba09-1f78d4361686': {'version…